## [NASA logs for July 1995](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
#!wget ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
#!gunzip NASA_access_log_Jul95.gz

## Load logs into a RDD

In [ ]:
sc = spark.sparkContext
nasa = sc.textFile('access_log_Jul95')

### How many non-empty lines?

In [ ]:
nasa.filter(lambda line: len(line) > 0).count()

### Create a function to parse Apache logs and, at the same time, counts malformed lines

In [ ]:
import re
# host unused user time method resource status size referer agent 
pattern = '^(\S+) \S+ (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+)\s?(\S+)?\s?(\S+)?" (\d{3}|-) (\d+|-)\s?"?([^"]*)"?\s?"?([^"]*)?"?$'
weblog = re.compile(pattern=pattern)

bad_line = spark.sparkContext.accumulator(0)

def process_logs(line):
    try:
        return weblog.findall(line)[0]
    except IndexError:
        bad_line.add(1)
        return ''

In [ ]:
print(nasa.take(1)[0])

In [ ]:
nasa.map(process_logs).filter(lambda line: line == '').count()

In [ ]:
bad_line.value

### Create a function to convert a tuple into a Row object

In [ ]:
import time, datetime

def parse_time(str_time):
    tt = time.strptime(str_time[:-6], "%d/%b/%Y:%H:%M:%S")
    return datetime.datetime(*tt[:6])

In [ ]:
from pyspark.sql import Row

no_size = spark.sparkContext.accumulator(0)

def to_int(value):
    try:
        value = int(value)
    except:
        value = -1
        no_size.add(1)
    return value

def log_to_row(line):
    fields = process_logs(line)
    if fields == '':
        fields = [''] * 10
        fields[2] = '01/Jan/1900:00:00:00 -0000'
        fields[6] = 0

    row = Row(host=fields[0], 
              user=fields[1], 
              time=parse_time(fields[2]),
              method=fields[3], 
              resource=fields[4], 
              protocol=fields[5],
              status=int(fields[6]),
              size=to_int(fields[7]),
              referer=fields[8], 
              agent=fields[9])
    return row

In [ ]:
log_to_row(nasa.take(1)[0])

### Create a DataFrame

In [ ]:
df = spark.createDataFrame(nasa.map(log_to_row))

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
no_size.value

In [ ]:
df.filter('size == -1').count()

### Which hosts were responsible for the largest amount of traffic?

In [ ]:
from pyspark.sql import functions as F

df.groupby('host').agg(F.sum('size').alias('total_size')).orderBy(F.desc('total_size')).show()

### Which resources were the most requested?

In [ ]:
df.groupby('resource').agg(F.count(F.lit(1)).alias('count')).orderBy(F.desc('count')).limit(20).toPandas()